<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/NLP/4-5-VectorDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chromadb openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.5 MB/s eta 0

In [2]:
!mkdir db

In [3]:
import chromadb
from chromadb.config import Settings

# Use PersistentClient and provide the path directly
client = chromadb.PersistentClient(path="db/")

In [4]:
collection = client.create_collection(name="Students")

In [5]:
student_info = """
Alexandra Thompson, a 19-year-old computer science sophomore with a 3.7 GPA,
is a member of the programming and chess clubs who enjoys pizza, swimming, and hiking
in her free time in hopes of working at a tech company after graduating from the University of Washington.
"""

club_info = """
The university chess club provides an outlet for students to come together and enjoy playing
the classic strategy game of chess. Members of all skill levels are welcome, from beginners learning
the rules to experienced tournament players. The club typically meets a few times per week to play casual games,
participate in tournaments, analyze famous chess matches, and improve members' skills.
"""

university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

add() function to add text data with metadata and unique IDs. After that, Chroma will automatically download the all-MiniLM-L6-v2 model to convert the text into embeddings and store it in the "Students" collection.

In [6]:
collection.add(
    documents = [student_info, club_info, university_info],
    metadatas = [{"source": "student info"},{"source": "club info"},{'source':'university info'}],
    ids = ["id1", "id2", "id3"]
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 45.5MiB/s]


In [7]:
results = collection.query(
    query_texts=["What is the student name?"],
    n_results=2
)


In [8]:

results

{'ids': [['id1', 'id2']],
 'distances': [[1.2946666564424738, 1.3954030668049473]],
 'metadatas': [[{'source': 'student info'}, {'source': 'club info'}]],
 'embeddings': None,
 'documents': [['\nAlexandra Thompson, a 19-year-old computer science sophomore with a 3.7 GPA,\nis a member of the programming and chess clubs who enjoys pizza, swimming, and hiking\nin her free time in hopes of working at a tech company after graduating from the University of Washington.\n',
   "\nThe university chess club provides an outlet for students to come together and enjoy playing\nthe classic strategy game of chess. Members of all skill levels are welcome, from beginners learning\nthe rules to experienced tournament players. The club typically meets a few times per week to play casual games,\nparticipate in tournaments, analyze famous chess matches, and improve members' skills.\n"]],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

## Embeddings


In [9]:
!pip install pandas openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.6 MB/s eta 0:00:00


In [10]:
from google.colab import userdata
import os

userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
api_key = os.getenv("OPENAI_API_KEY")

In [11]:
from chromadb.utils import embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=api_key, # Pass the api key to OpenAIEmbeddingFunction
                model_name="text-embedding-ada-002"
            )
students_embeddings = openai_ef([student_info, club_info, university_info])
print(students_embeddings)

[array([-0.012634  ,  0.00815973,  0.01234951, ..., -0.0158022 ,
       -0.00304373, -0.01894453]), array([-0.00612561, -0.01127163,  0.01216099, ..., -0.02522422,
        0.0189085 , -0.03663119]), array([ 0.00749402,  0.0026733 , -0.00870859, ..., -0.0142618 ,
        0.0110939 , -0.01364825])]


In [12]:
collection2 = client.get_or_create_collection(name="Students2")

collection2.add(
    embeddings = students_embeddings,
    documents = [student_info, club_info, university_info],
    metadatas = [{"source": "student info"},{"source": "club info"},{'source':'university info'}],
    ids = ["id1", "id2", "id3"]
)

In [13]:
collection2 = client.get_or_create_collection(name="Students2",embedding_function=openai_ef)

collection2.add(
    documents = [student_info, club_info, university_info],
    metadatas = [{"source": "student info"},{"source": "club info"},{'source':'university info'}],
    ids = ["id1", "id2", "id3"]
)

In [14]:
results = collection2.query(
    query_texts=["What is the student name?"],
    n_results=2
)

results

{'ids': [['id1', 'id3']],
 'distances': [[0.44034465108690896, 0.4905111521657062]],
 'metadatas': [[{'source': 'student info'}, {'source': 'university info'}]],
 'embeddings': None,
 'documents': [['\nAlexandra Thompson, a 19-year-old computer science sophomore with a 3.7 GPA,\nis a member of the programming and chess clubs who enjoys pizza, swimming, and hiking\nin her free time in hopes of working at a tech company after graduating from the University of Washington.\n',
   '\nThe University of Washington, founded in 1861 in Seattle, is a public research university\nwith over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.\nAs the flagship institution of the six public universities in Washington state,\nUW encompasses over 500 buildings and 20 million square feet of space,\nincluding one of the largest library systems in the world.\n']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

## 유사도 검색이 개선되어 이제 동아리 대신 대학에 대한 정보를 반환합니다. 또한, 벡터 간의 거리가 기본 임베딩 모델보다 더 낮아졌으며, 이는 긍정적인 결과입니다.

## Updating and Removing Data

In [15]:
collection2.update(
    ids=["id1"],
    documents=["Kristiane Carina, a 19-year-old computer science sophomore with a 3.7 GPA"],
    metadatas=[{"source": "student info"}],
)

In [16]:
results = collection2.query(
    query_texts=["What is the student name?"],
    n_results=2
)

results

{'ids': [['id1', 'id3']],
 'distances': [[0.3755055002881767, 0.4905111521657062]],
 'metadatas': [[{'source': 'student info'}, {'source': 'university info'}]],
 'embeddings': None,
 'documents': [['Kristiane Carina, a 19-year-old computer science sophomore with a 3.7 GPA',
   '\nThe University of Washington, founded in 1861 in Seattle, is a public research university\nwith over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.\nAs the flagship institution of the six public universities in Washington state,\nUW encompasses over 500 buildings and 20 million square feet of space,\nincluding one of the largest library systems in the world.\n']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [17]:
collection2.delete(ids = ['id1'])


results = collection2.query(
    query_texts=["What is the student name?"],
    n_results=2
)

results

{'ids': [['id3', 'id2']],
 'distances': [[0.4905111521657062, 0.5320566977669932]],
 'metadatas': [[{'source': 'university info'}, {'source': 'club info'}]],
 'embeddings': None,
 'documents': [['\nThe University of Washington, founded in 1861 in Seattle, is a public research university\nwith over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.\nAs the flagship institution of the six public universities in Washington state,\nUW encompasses over 500 buildings and 20 million square feet of space,\nincluding one of the largest library systems in the world.\n',
   "\nThe university chess club provides an outlet for students to come together and enjoy playing\nthe classic strategy game of chess. Members of all skill levels are welcome, from beginners learning\nthe rules to experienced tournament players. The club typically meets a few times per week to play casual games,\nparticipate in tournaments, analyze famous chess matches, and improve members' skills.\n"]],
 'ur

## Collection Management

In [18]:
vector_collections = client.create_collection("vectordb")


vector_collections.add(
    documents=["This is Chroma DB CheatSheet",
               "This is Chroma DB Documentation",
               "This document Chroma JS API Docs"],
    metadatas=[{"source": "Chroma Cheatsheet"},
    {"source": "Chroma Doc"},
    {'source':'JS API Doc'}],
    ids=["id1", "id2", "id3"]
)

In [19]:
vector_collections.count()

3

In [20]:
vector_collections.get() # To view all the records from the collection

{'ids': ['id1', 'id2', 'id3'],
 'embeddings': None,
 'metadatas': [{'source': 'Chroma Cheatsheet'},
  {'source': 'Chroma Doc'},
  {'source': 'JS API Doc'}],
 'documents': ['This is Chroma DB CheatSheet',
  'This is Chroma DB Documentation',
  'This document Chroma JS API Docs'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [21]:
vector_collections.modify(name="chroma_info")

# list all collections
client.list_collections()

[Collection(id=47d41ba3-70f7-4386-88c9-65a312971f7b, name=chroma_info),
 Collection(id=4f4ffa7f-f8b5-41d0-b294-e36fee8bdf7d, name=Students),
 Collection(id=fe5caf49-db5c-400c-a264-4854adf5e122, name=Students2)]

In [25]:
client.delete_collection(name="chroma_info")
client.list_collections()

[Collection(id=4f4ffa7f-f8b5-41d0-b294-e36fee8bdf7d, name=Students),
 Collection(id=fe5caf49-db5c-400c-a264-4854adf5e122, name=Students2)]

In [ ]:
client.reset()
client.list_collections()